# OpenAI Function Calling

In [3]:
import os
import openai

OPENAI_API_KEY_PATH = '/home/loc/Documents/OPENAI_API_KEY.txt'

with open(OPENAI_API_KEY_PATH) as f:
    OPENAI_API_KEY = f.read().strip()
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY